In [78]:
import pandas as pd
import numpy as np
import torch
from importlib import reload
# import torch 
import CIR_
from CIR_ import CIR
import CIR2_
from CIR2_ import CIR2
reload(CIR_)
reload(CIR2_)

import sys
import numpy
numpy.set_printoptions(suppress=True, threshold=sys.maxsize)

X = pd.read_csv('foregroundX.csv')
Y = pd.read_csv('foregroundY.csv')
Xt = pd.read_csv('backgroundX.csv')
Yt = pd.read_csv('backgroundY.csv')

O = pd.read_csv('output.csv')
O = pd.DataFrame(O)
O = O.iloc[:, 1:]
O = O.to_numpy()
output = O @ np.transpose(O)
# output = torch.from_numpy(output)
# print("Correct output....")
# print(output)

# # n = pd.read_csv('test - noIndex.csv')

# X = pd.read_csv('test - fX.csv')
# Y = pd.read_csv('test - fY.csv')
# Xt = pd.read_csv('test - bX.csv')
# Yt = pd.read_csv('test - fY.csv')

X = X.iloc[:, 1:]
Xt = Xt.iloc[:, 1:]
Y = Y.iloc[:, 1:]
Yt = Yt.iloc[:, 1:]

# print("CIR output............")
# t = CIR(X, Y, Xt, Yt, 0.0001, 2)
# print(torch.subtract(output, t))

# print(torch.subtract(output, t))

t1 = CIR2(X, Y, Xt, Yt, 0.0001, 2)
print(t1)
# print(np.subtract(output, t1))
# print(output)


[[ 0.404789827815766 -0.100932801126461 -0.01496908870894
   0.056006877411581 -0.016428220860815 -0.017059978591601
  -0.002677611795282 -0.034138083388007  0.00633203198154
   0.006531203997728 -0.051751798180825  0.002217859527477
  -0.009968517000075 -0.003824318067797 -0.031420179449275
  -0.003983353136941 -0.013506696662411  0.113032686687039
  -0.045403683820404 -0.062142410556268 -0.042814554204441
   0.037850503985607  0.011324278344791 -0.029769974910778
   0.078767975368576 -0.197872596224093 -0.008853557503379
  -0.017091992014807  0.02105175078389   0.017872587387886
  -0.015067704353406 -0.000933658025112 -0.069873102371851
  -0.050000949254736 -0.1040261791927   -0.06824838722826
  -0.026275910424075 -0.017191183118395  0.001155136552505
   0.050603247782635 -0.012440855804995 -0.00176245938425
   0.027967086164583  0.048919584298373 -0.080710922239706
   0.031629195564539  0.018157557338323  0.064142833866101
   0.184398057680707  0.010644349237775 -0.083108105876053
 

In [ ]:
# t = pd.read_csv('test-fX.csv')
# t = pd.DataFrame(t)
# if t.iloc[:, 0].dtypes != t.iloc[:, 1].dtypes:
#     t = t.iloc[:, 1:]
